<h1>Spark SQL, DataFrames and Datasets Guide</h1>
<ul>
  <li>Overview<ul>
      <li>SQL</li>
      <li>Datasets and DataFrames</li>
    </ul>
  </li>
  <li>Getting Started<ul>
      <li>Starting Point: SparkSession</li>
      <li>Creating DataFrames</li>
      <li>Untyped Dataset Operations(aka DataFrame Operations)</li>
      <li>Running SQL Queries Programmatically</li>
      <li>Creating Datasets</li>
      <li>Interoperating with RDDs<ul>
          <li>Inferring the Schema Using Reflection</li>
          <li>Programmatically Specifying the Schema</li>
        </ul>
      </li>
    </ul>
  </li>
  <li>Data Sources<ul>
      <li>Generic Load/Save Functions<ul>
          <li>Manually Specifying Options</li>
          <li>Run SQL on files directly</li>
          <li>Save Modes</li>
          <li>Saving to Persistent Tables</li>
        </ul>
      </li>
      <li>Parquet Files<ul>
          <li>Loading Data Programmatically</li>
          <li>Partition Discovery</li>
          <li>Schema Merging</li>
          <li>Hive metastore Parquet table conversion<ul>
              <li>Hive/Parquet Schema Reconciliation</li>
              <li>Metadata Refreshing</li>
            </ul>
          </li>
          <li>Configuration</li>
        </ul>
      </li>
      <li>JSON Datasets</li>
      <li>Hive Tables<ul>
          <li>Interacting with Different Versions of Hive Metastore</li>
        </ul>
      </li>
      <li>JDBC To Other Databases</li>
      <li>Troubleshooting</li>
    </ul>
  </li>
  <li>Performance Tuning<ul>
      <li>Caching Data In Memory</li>
      <li>Other Configuration Options</li>
    </ul>
  </li>
  <li>Distributed SQL Engine<ul>
      <li>Running the Thrift JDBC/ODBC server</li>
      <li>Running the Spark SQL CLI</li>
    </ul>
  </li>
  <li>Migration Guide<ul>
      <li>Upgrading From Spark SQL 1.6 to 2.0</li>
      <li>Upgrading From Spark SQL 1.5 to 1.6</li>
      <li>Upgrading From Spark SQL 1.4 to 1.5</li>
      <li>Upgrading from Spark SQL 1.3 to 1.4 <ul>
          <li>DataFrame data reader/writer interface</li>
          <li>DataFrame.groupBy retains grouping columns</li>
          <li>Behavior change on DataFrame.withColumn</li>
        </ul>
      </li>
      <li>Upgrading from Spark SQL 1.0-1.2 to 1.3<ul>
          <li>Rename of SchemaRDD to DataFrame</li>
          <li>Unification of the Java and Scala APIs</li>
          <li>Isolation of Implicit Conversions and Removal of dsl Package (Scala-only)</li>
          <li>Removal of the type aliases in org.apache.spark.sql for DataType (Scala-only)</li>
          <li>UDF Registration Moved to <code>sqlContext.udf</code> (Java &amp; Scala)</li>
          <li>Python DataTypes No Longer Singletons</li>
        </ul>
      </li>
      <li>Compatibility with Apache Hive<ul>
          <li>Deploying in Existing Hive Warehouses</li>
          <li>Supported Hive Features</li>
          <li>Unsupported Hive Functionality</li>
        </ul>
      </li>
    </ul>
  </li>
  <li>Reference    <ul>
      <li>Data Types</li>
      <li>NaN Semantics</li>
    </ul>
  </li>
</ul>

<h1>Overview</h1>
<p>Spark SQL is a Spark module for structured data processing. Unlike the basic Spark RDD API, the interfaces provided
by Spark SQL provide Spark with more information about the structure of both the data and the computation being performed. Internally,Spark SQL uses this extra information to perform extra optimizations. There are several ways to
interact with Spark SQL including SQL and the Dataset API. When computing a result the same execution engine is used, independent of which API/language you are using to express the computation. This unification(统一，联合; 一致;
) means that developers can easily switch back and forth between different APIs based on which provides the most natural way to express a given transformation.</p>
<p>All of the examples on this page use sample data included in the Spark distribution and can be run in the <code>spark-shell</code>, <code>pyspark</code> shell, or <code>sparkR</code> shell.</p>

<h2>SQL</h2>
<p>One use of Spark SQL is to execute SQL queries.Spark SQL can also be used to read data from an existing Hive installation. For more on how to configure this feature, please refer to the <strong>Hive Tables</strong> section. When running SQL from within another programming language the results will be returned as a <strong>Dataset/DataFrame</strong>.You can also interact with the SQL interface using the <strong>command-line</strong> or over <strong>JDBC/ODBC</strong>.</p>

<h2>Datasets and DataFrames</h2>

<p>A Dataset is a distributed collection of data.Dataset is a new interface added in Spark 1.6 that provides the benefits of RDDs (strong typing, ability to use powerful lambda functions) with the benefits of Spark SQL&#8217;s optimized execution engine. A Dataset can be <strong>constructed</strong> from JVM objects and then
manipulated using functional transformations (<code>map</code>, <code>flatMap</code>, <code>filter</code>, etc.).
The Dataset API is available in <a href="http://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Dataset">Scala</a> and
<a href="http://spark.apache.org/docs/latest/api/java/index.html?org/apache/spark/sql/Dataset.html">Java</a>. Python does not have the support for the Dataset API. But due to Python&#8217;s dynamic nature,many of the benefits of the Dataset API are already available (i.e. you can access the field of a row by name naturally<code>row.columnName</code>). The case for R is similar.</p>

<p>A DataFrame is a <em>Dataset</em> organized into named columns. It is conceptually equivalent to a table in a relational database or a data frame in R/Python, but with richer optimizations under the hood(在幕后). DataFrames can be constructed from a wide array of <a href="http://spark.apache.org/docs/latest/sql-programming-guide.html#data-sources">sources</a> such as: <u>structured data files, tables in Hive, external databases, or existing RDDs</u>.The DataFrame API is available in Scala,Java, <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame">Python</a>, and <a href="http://spark.apache.org/docs/latest/api/R/index.html">R</a>.In Scala and Java, a DataFrame is represented by a Dataset of <code>Row</code>s.In <a href="http://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Dataset">the Scala API</a>, <code>DataFrame</code> is simply a type alias of <code>Dataset[Row]</code>.While, in <a href="http://spark.apache.org/docs/latest/api/java/index.html?org/apache/spark/sql/Dataset.html">Java API</a>, users need to use <code>Dataset&lt;Row&gt;</code> to represent a <code>DataFrame</code>.</p>

<p>Throughout this document, we will often refer to Scala/Java Datasets of <code>Row</code>s as DataFrames.</p>

<h1>Getting Started</h1>
<h2>Starting Point: SparkSession</h2>
<p>The entry point into(入口点) all functionality in Spark is the <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.SparkSession"><code>SparkSession</code></a> class. To create a basic <code>SparkSession</code>, just use <code>SparkSession.builder</code>:</p>

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession\
.builder\
.appName("PythonSQL")\
.config("spark.some.config.option", 'some-value')\
.getOrCreate()

<p><code>SparkSession</code> in Spark 2.0 provides builtin support for Hive features including the ability to write queries using HiveQL, access to Hive UDFs(User defined Function,用户自定义函数), and the ability to read data from Hive tables.To use these features, you do not need to have an existing Hive setup.</p>

<h2 id="creating-dataframes">Creating DataFrames</h2>
<p>With a <code>SparkSession</code>, applications can create DataFrames from an <a href="http://spark.apache.org/docs/latest/sql-programming-guide.html#interoperating-with-rdds">existing <code>RDD</code></a>,from a Hive table, or from <a href="http://spark.apache.org/docs/latest/sql-programming-guide.html#data-sources">Spark data sources</a>.</p>

<p>As an example, the following creates a DataFrame based on the content of a JSON file:</p>

In [2]:
# spark is an existing SparkSession
df = spark.read.json("resources/people.json")
# Displays the content of the DataFrame to stdout
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



<h2>Untyped Dataset Operations (aka DataFrame Operations)</h2>
<p>DataFrames provide a domain-specific language(DSL:领域特定语言) for structured data manipulation in <a href="https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Dataset">Scala</a>, <a href="https://spark.apache.org/docs/latest/api/java/index.html?org/apache/spark/sql/Dataset.html">Java</a>, <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame">Python</a> and <a href="https://spark.apache.org/docs/latest/api/R/DataFrame.htmll">R</a>.</p>
<p>As mentioned above, in Spark 2.0, DataFrames are just Dataset of <code>Row</code>s in Scala and Java API. These operations are also referred as “untyped transformations” in contrast to “typed transformations” come with strongly typed(强类型) Scala/Java Datasets.</p>
<p>Here we include some basic examples of structured data processing using Datasets:</p>
<p>In Python it’s possible to access a DataFrame’s columns either by attribute (<code>df.age</code>) or by indexing (<code>df['age']</code>). While the former(前者) is convenient for interactive data exploration, users are highly encouraged to use the latter form, which is future proof and won’t break with column names that are also attributes on the DataFrame class.</p>

In [3]:
# spark, df are from the previous example
# Print the schema in a tree format
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [4]:
# Select only the "name" column
df.select("name").show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+



In [5]:
# Select everybody, but increment the age by 1
df.select(df["name"], df["age"] + 1).show()

+-------+---------+
|   name|(age + 1)|
+-------+---------+
|Michael|     null|
|   Andy|       31|
| Justin|       20|
+-------+---------+



In [6]:
# Select people older than 21
df.filter(df['age'] > 21).show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



In [7]:
# Count people by age
df.groupBy('age').count().show()

+----+-----+
| age|count|
+----+-----+
|  19|    1|
|null|    1|
|  30|    1|
+----+-----+



<p>For a complete list of the types of operations that can be performed on a DataFrame refer to the <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame">API Documentation</a>.</p>
<p>In addition to simple column references and expressions, DataFrames also have a rich library of functions including string manipulation, date arithmetic, common math operations and more. The complete list is available in the <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions">DataFrame Function Reference</a>.</p>
<h2>Running SQL Queries Programmatically(编码的方式运行SQL查询)</h2>
<ul>
<li><b>Scala</b></li><li><b>Java</b></li>
<li><b>Python</b></li><li><b>R</b></li>
</ul>
<p>The <code>sql</code> function on a <code>SparkSession</code> enables applications to run SQL queries programmatically and returns the result as a <code>DataFrame</code>.</p>

<dl>
<dt>
<a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=createorreplacetempview#pyspark.sql.DataFrame.createOrReplaceTempView" ><tt>createOrReplaceTempView</tt><big>(</big><em>name</em><big>)</big></a></dt>
<dd><p>Creates or replaces a temporary view with this DataFrame.</p>
<p>The lifetime of this temporary table is tied to the <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=createorreplacetempview#pyspark.sql.SparkSession" target="_blank"><tt>SparkSession</tt></a>
that was used to create this <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=createorreplacetempview#pyspark.sql.DataFrame"><tt>DataFrame</tt></a>.</p>
</dd></dl>

In [8]:
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("people")
sqlDF = spark.sql("select * from people")
sqlDF.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



<h2>Creating Datasets(仅支持Scala,Java)</h2>
<p>Datasets are similar to RDDs, however, instead of using Java serialization or Kryo they use
a specialized <a href="https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Encoder">Encoder</a> to serialize the objects
for processing or transmitting over the network. While both encoders and standard serialization are
responsible for turning an object into bytes, encoders are code generated dynamically and use a format
that allows Spark to perform many operations like filtering, sorting and hashing without deserializing
the bytes back into an object.</p>

<h2>Interoperating with(互操作) RDDs</h2>

<p>Spark SQL supports two different methods for converting existing RDDs into Datasets. The first method uses reflection(反射) to infer(推断; 猜想) the schema of an RDD that contains specific types of objects. This reflection based approach leads to more concise code(简洁的代码) and works well when you already know the schema while writing your Spark application.</p>
<p>The second method for creating Datasets is through a programmatic interface that allows you to construct a schema and then apply it to an existing RDD. While this method is more verbose(冗长的，啰唆的，累赘的;), it allows you to construct Datasets when the columns and their types are not known until runtime.</p>

<h3>Inferring the Schema Using Reflection</h3>
<p>Spark SQL can convert an RDD of <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=row#pyspark.sql.Row" target="_blank">Row</a> objects to a DataFrame, inferring the datatypes. Rows are constructed by passing a list of key/value pairs as kwargs to the Row class. The keys of this list define the column names of the table, and the types are inferred by sampling the whole datase, similar to the inference that is performed on JSON files.</p>

In [9]:
from pyspark.sql import Row
# Load a text file and convert each line to a Row.
lines = sc.textFile("resources/people.txt")
lines.collect()

['Michael, 29', 'Andy, 30', 'Justin, 19']

In [10]:
parts = lines.map(lambda line: line.split(","))
parts.collect()

[['Michael', ' 29'], ['Andy', ' 30'], ['Justin', ' 19']]

In [11]:
people = parts.map(lambda people: Row(name=people[0], age=int(people[1])))
people.collect()

[Row(age=29, name='Michael'),
 Row(age=30, name='Andy'),
 Row(age=19, name='Justin')]

In [12]:
# Infer the schema, and register the DataFrame as a table.
schemaPeople = spark.createDataFrame(people)
schemaPeople.createOrReplaceTempView("people")
# SQL can be run over DataFrames that have been registered as a table.
teenagers = spark.sql("SELECT name FROM people WHERE age >= 13 AND age <= 19")
# The results of SQL queries are Dataframe objects.
# rdd returns the content as an :class:`pyspark.RDD` of :class:`Row`.
teenNames = teenagers.rdd.map(lambda p: "Name: " + p.name).collect()
for name in teenNames:
    print(name)

Name: Justin


<h3>Programmatically Specifying the Schema</h3>
<p>When case classes cannot be defined ahead of time (for example,the structure of records is encoded in a string, or a text dataset will be parsed and fields will be projected differently for different users),a <code>DataFrame</code> can be created programmatically with three steps.</p>
<ol>
  <li>Create an RDD of <code>Row</code>s from the original RDD;</li>
  <li>Create the schema represented by a <a  href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=structtype#pyspark.sql.types.StructType" target="_blank"><code>StructType</code></a> matching the structure of
<code>Row</code>s in the RDD created in Step 1.</li>
  <li>Apply the schema to the RDD of <code>Row</code>s via <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=createdataframe#pyspark.sql.SparkSession.createDataFrame" target="_blank"><code>createDataFrame</code></a> method provided by <code>SparkSession</code>.</li>
</ol>
<p>For example:</p>

In [13]:
# Import data types
from pyspark.sql.types import *
# Load a text file and convert each line to a Row.
lines = sc.textFile("resources/people.txt")
parts = lines.map(lambda line: line.split(","))
# Each line is converted to a tuple.
people = parts.map(lambda people: (people[0], people[1].strip()))
# The schema is encoded in a string.
schemaString = "name age"
fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
print("fields:", fields, '\n')
schema = StructType([StructField("name", StringType(), True), StructField("age", StringType(), True)])
print("schema:", schema, '\n')
# Apply the schema to the RDD.
schemaPeople = spark.createDataFrame(people, schema)
# Creates a temporary view using the DataFrame
schemaPeople.createOrReplaceTempView("people")
# SQL can be run over DataFrames that have been registered as a table.
results = spark.sql("SELECT name FROM people")
results.show()

fields: [StructField(name,StringType,true), StructField(age,StringType,true)] 

schema: StructType(List(StructField(name,StringType,true),StructField(age,StringType,true))) 

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+



<h1>Data Sources</h1>

<p>Spark SQL supports operating on a variety of data sources through the DataFrame interface.A DataFrame can be operated on using relational transformations and can also be used to create a temporary view.<strong>Registering a DataFrame as a temporary view allows you to run SQL queries over its data</strong>. This section describes the general methods for loading and saving data using the Spark Data Sources and then goes into specific options that are available for the built-in data sources.</p>

<h2>Generic Load/Save Functions</h2>

<p>In the simplest form, the default data source(<code>parquet</code> <u>unless otherwise configured</u>(除非另外配置) by <code>spark.sql.sources.default</code>) will be used for all operations.</p>

In [14]:
df = spark.read.load("resources/users.parquet")
df.select("name","favorite_color").write.save("resources/namesAndFavColors.parquet")

In [15]:
df.show()

+------+--------------+----------------+
|  name|favorite_color|favorite_numbers|
+------+--------------+----------------+
|Alyssa|          null|  [3, 9, 15, 20]|
|   Ben|           red|              []|
+------+--------------+----------------+



<h3>Manually Specifying Options</h3>

<p>You can also manually specify the data source that will be used along with any extra options that you would like to pass to the data source. Data sources are specified by their fully qualified name (i.e., <code>org.apache.spark.sql.parquet</code>), but for built-in sources you can also use their short names (<code>json</code>, <code>parquet</code>, <code>jdbc</code>). <strong>DataFrames loaded from any data source type can be converted into other types using this syntax</strong>.</p>

In [16]:
df = spark.read.load("resources/people.json", format="json")
df.select("name", "age").write.save("resources/namesAndAges.parquet", format="parquet")

<h3>Run SQL on files directly</h3>

<p>Instead of using read API to load a file into DataFrame and query it, you can also query that file directly with SQL.</p>

In [17]:
df = spark.sql("SELECT * FROM parquet.`resources/users.parquet`")
df.show()

+------+--------------+----------------+
|  name|favorite_color|favorite_numbers|
+------+--------------+----------------+
|Alyssa|          null|  [3, 9, 15, 20]|
|   Ben|           red|              []|
+------+--------------+----------------+



<h3>Save Modes</h3>

<p>Save operations can optionally take a <code>SaveMode</code>, that specifies how to handle existing data if
present. <font color="red"><strong>It is important to realize that these save modes do not utilize(利用;运用;使用;应用) any locking and are not atomic. Additionally, when performing an <code>Overwrite</code>, the data will be deleted before writing out the new data</strong>.</font></p>

<table>
<tr><th>Scala/Java</th><th>Any Language</th><th>Meaning</th></tr>
<tr>
  <td><code>SaveMode.ErrorIfExists</code> (default)</td>
  <td><code>"error"</code> (default)</td>
  <td>
    When saving a DataFrame to a data source, if data already exists,an exception is expected to be thrown.
  </td>
</tr>
<tr>
  <td><code>SaveMode.Append</code></td>
  <td><code>"append"</code></td>
  <td>
    When saving a DataFrame to a data source, if data/table already exists,contents of the DataFrame are expected to be appended to existing data.
  </td>
</tr>
<tr>
  <td><code>SaveMode.Overwrite</code></td>
  <td><code>"overwrite"</code></td>
  <td>
    Overwrite mode means that when saving a DataFrame to a data source,if data/table already exists, existing data is expected to be overwritten by the contents of the DataFrame.
  </td>
</tr>
<tr>
  <td><code>SaveMode.Ignore</code></td>
  <td><code>"ignore"</code></td>
  <td>
    Ignore mode means that when saving a DataFrame to a data source, if data already exists,the save operation is expected to not save the contents of the DataFrame and to not change the existing data. This is similar to a <code>CREATE TABLE IF NOT EXISTS</code> in SQL.
  </td>
</tr>
</table>

<h3>Saving to Persistent Tables</h3>

<p><code>DataFrames</code> can also be saved as persistent tables into Hive metastore using the <strong><code>saveAsTable</code></strong>
command. Notice existing Hive deployment is not necessary to use this feature. Spark will create a default local Hive metastore (using Derby) for you. Unlike the <strong><code>createOrReplaceTempView</code></strong> command,<strong><code>saveAsTable</code></strong> will materialize(实现) the contents of the DataFrame and create a pointer to the data in the Hive metastore. Persistent tables will still exist even after your Spark program has restarted, as long as you maintain your connection to the same metastore. A DataFrame for a persistent table can be created by calling the <strong><code>table</code></strong> method on a <strong><code>SparkSession</code></strong> with the name of the table.</p>

<p>By default <strong><code>saveAsTable</code></strong> will create a &#8220;managed table&#8221;, meaning that the location of the data will be controlled by the metastore. Managed tables will also have their data deleted automatically when a table is dropped.</p>
<p>在Hive上有两种类型的表，一种是Managed Table，另一种是External Table。它俩的主要区别在于：当我们drop表时，Managed Table会同时删去data和meta data，而External Table只会删meta data。</p>
<h2>Parquet Files</h2>

<p><a href="http://parquet.io">Parquet</a> is a columnar format that is supported by many other data processing systems.Spark SQL provides support for both reading and writing Parquet files that automatically preserves(保留；保护；保存；维护) the schema of the original data. When writing Parquet files, all columns are automatically converted to be nullable(允许空值) for compatibility reasons(出于兼容方面的考虑).</p>

<h3>Loading Data Programmatically</h3>

<p>Using the data from the above example:</p>

In [18]:
peopleDF = spark.read.json("resources/people.json")

# DataFrames can be saved as Parquet files, maintaining the schema information.
peopleDF.write.parquet("resources/people.parquet")

# Read in the Parquet file created above.
# Parquet files are self-describing so the schema is preserved.
# The result of loading a parquet file is also a DataFrame.
parquetFile = spark.read.parquet("resources/people.parquet")
# Parquet files can also be used to create a temporary view and then used in SQL statements.
parquetFile.createOrReplaceTempView("parquetFile")
teenagers = spark.sql("SELECT name FROM parquetFile WHERE age >= 13 AND age <= 19")
teenagers.show()

+------+
|  name|
+------+
|Justin|
+------+



<h3>Partition Discovery(解析分区信息)</h3>
<p>Table partitioning(表分区) is a common optimization approach(优化方法) used in systems like Hive. In a partitioned table(分区表), data are usually stored in different directories, with partitioning column values encoded(编码) in the path of each partition directory. The Parquet data source is now able to discover and infer partitioning information automatically. For example, we can store all our previously used population data into a partitioned table using the following directory structure, with two extra columns, <code>gender</code>(性别) and <code>country</code>(国家) as partitioning columns:</p>
<p>类似hive的分区表，在分区表中数据会分开存储在不同的文件夹，区分的标准是分区字段，现在parquet的数据源可以自动的发现分区信息.</p>
<pre><code>path
└── to
    └── table
        ├── gender=male
        │   ├── ...
        │   │
        │   ├── country=US
        │   │   └── data.parquet
        │   ├── country=CN
        │   │   └── data.parquet
        │   └── ...
        └── gender=female
            ├── ...
            │
            ├── country=US
            │   └── data.parquet
            ├── country=CN
            │   └── data.parquet
            └── ...</code></pre>
<p>By passing <code>path/to/table</code> to either <code>SparkSession.read.parquet</code> or <code>SparkSession.read.load</code>, Spark SQL will automatically extract the partitioning information from the paths.Now the schema of the returned DataFrame becomes:</p>
<p>通过传递path/to/table给SparkSession.read.parquet或SparkSession.read.load，Spark SQL将自动解析分区信息。返回的DataFrame的Schema如下：</p>
<pre><code>root
|-- name: string (nullable = true)
|-- age: long (nullable = true)
|-- gender: string (nullable = true)
|-- country: string (nullable = true)</code></pre>
<p>Notice that the data types of the partitioning columns are automatically inferred. Currently, numeric data types and string type are supported. Sometimes users may not want to automatically infer the data types of the partitioning columns. For these use cases, the automatic type inference can be configured by <code>spark.sql.sources.partitionColumnTypeInference.enabled</code>, which is default to <code>true</code>. When type inference is disabled, string type will be used for the partitioning columns.</p>
<p>需要注意的是，数据的分区列的数据类型是自动解析的。当前，支持数值类型和字符串类型。自动解析分区类型的参数为：spark.sql.sources.partitionColumnTypeInference.enabled，默认值为true。如果想关闭该功能，直接将该参数设置为disabled。此时，分区列数据格式将被默认设置为string类型，不再进行类型解析。</p>
<p>Starting from Spark 1.6.0, partition discovery only finds partitions under the given paths by default. For the above example, if users pass <code>path/to/table/gender=male</code> to either <code>SparkSession.read.parquet</code> or <code>SparkSession.read.load</code>, <code>gender</code> will not be considered as a partitioning column. If users need to specify the base path that partition discovery should start with, they can set <code>basePath</code> in the data source options. For example, when <code>path/to/table/gender=male</code> is the path of the data and users set <code>basePath</code> to <code>path/to/table/</code>, <code>gender</code> will be a partitioning column.</p>
<p>从1.6版本开始，如果用了分区路径，那么会丢失用于分区路径的那个字段，只能通过basedir拿到，而把分区属性变成分区字段.</p>

In [20]:
spark.read.parquet("resources/people.parquet").show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



<dl>
<a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=partitionby#pyspark.sql.DataFrameWriter.partitionBy" target="_blank">
<dt>
<tt>partitionBy</tt><big>(</big><em>*cols</em><big>)</big></dt></a>
<dd><p>Partitions the output by the given columns on the file system.</p>
<p>If specified, the output is laid out on the file system similar
to Hive’s partitioning scheme.</p>
<table rules="none">
<colgroup><col class="field-name">
<col class="field-body">
</colgroup><tbody valign="top">
<tr class="field-odd field"><th class="field-name">Parameters:</th><td class="field-body"><strong>cols</strong> – name of columns</td>
</tr>
</tbody>
</table>
<div class="highlight-python"><div class="highlight"><pre><span class="gp">&gt;&gt;&gt; </span><span class="n">df</span><span class="o">.</span><span class="n">write</span><span class="o">.</span><span class="n"><span class="highlighted">partitionBy</span></span><span class="p">(</span><span class="s">'year'</span><span class="p">,</span> <span class="s">'month'</span><span class="p">)</span><span class="o">.</span><span class="n">parquet</span><span class="p">(</span><span class="n">os</span><span class="o">.</span><span class="n">path</span><span class="o">.</span><span class="n">join</span><span class="p">(</span><span class="n">tempfile</span><span class="o">.</span><span class="n">mkdtemp</span><span class="p">(),</span> <span class="s">'data'</span><span class="p">))</span>
</pre></div>
</div>
</dd></dl>
<p>从这个函数我们可以看出按照我们给出的字段完成了一个partition的过程，接受一个多参数的字段列表.</p>
<font color="red">关于Partition Discovery更多详情请参考<a href="http://blog.csdn.net/cjuexuan/article/details/51165310" target="_blank">这篇博文</a>。</font>

In [22]:
spark.read.json("resources/people.json").write.partitionBy("name").parquet("resources/peoplePartitionBy.parquet")

<h3>查看输出路径下的目录：</h3>
<pre><code >_SUCCESS      name=Michael        name=Justin        name=Andy</code></pre>

<p>我们可以发现按照我们的分区规则，在输出路径上建立了三个文件夹，分别为name=Michael,name=Justin,name=Andy</p>
<h3>读取：</h3>

In [23]:
spark.read.parquet("resources/peoplePartitionBy.parquet/name=Andy").show()

+---+
|age|
+---+
| 30|
+---+



In [24]:
spark.read.parquet("resources/peoplePartitionBy.parquet/name=Michael").show()

+----+
| age|
+----+
|null|
+----+



In [25]:
spark.read.parquet("resources/peoplePartitionBy.parquet/name=Justin").show()

+---+
|age|
+---+
| 19|
+---+



In [26]:
spark.read.parquet("resources/peoplePartitionBy.parquet").show()

+----+-------+
| age|   name|
+----+-------+
|  30|   Andy|
|  19| Justin|
|null|Michael|
+----+-------+



<dl>
<dt>
<tt>filter</tt><big>(</big><em>condition</em><big>)</big><a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=filter#pyspark.sql.DataFrame.filter" target="_blank">¶</a></dt>
<dd><p>Filters rows using the given condition.</p>
<p><a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=filter#pyspark.sql.DataFrame.where"><tt>where()</tt></a> is an alias for <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=filter#pyspark.sql.DataFrame.filter"><tt>filter()</tt></a>.</p>
<table frame="void" rules="none">
<colgroup><col>
<col>
</colgroup><tbody valign="top">
<tr><th>Parameters:</th><td><strong>condition</strong> – a <a  href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=filter#pyspark.sql.Column" ><tt>Column</tt></a> of <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=filter#pyspark.sql.types.BooleanType" ><tt>types.BooleanType</tt></a>
or a string of SQL expression.</td>
</tr>
</tbody>
</table>
<pre>&gt;&gt;&gt; df.filter(.age&gt;3).collect()
[Row(age=5, name=u'Bob')]
&gt;&gt;&gt; df.where(df.age==2).collect()
[Row(age=2, name=u'Alice')]
</pre>
<pre>&gt;&gt;&gt; df.filter("age &gt; 3").collect()
[Row(age=5, name=u'Bob')]
&gt;&gt;&gt; df.where("age = 2").collect()
[Row(age=2, name=u'Alice')]
</pre>
</dd></dl>

In [43]:
df = spark.read.parquet("resources/peoplePartitionBy.parquet")
df.filter(df.name == "Andy").collect()

[Row(age=30, name='Andy')]

In [45]:
spark.read.parquet("resources/peoplePartitionBy.parquet").filter('name="Andy"').show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



<h3>Schema Merging</h3>
<p>Like ProtocolBuffer, Avro, and Thrift, Parquet also supports schema evolution(演变; 进化; 发展;). Users can start with a simple schema, and gradually add more columns to the schema as needed. In this way, users may end up with multiple Parquet files with different but mutually compatible(相互兼容) schemas. The Parquet data source is now able to automatically detect this case and merge schemas of all these files.</p>

<p>Since schema merging is a relatively expensive operation, and is not a necessity in most cases, we turned it off by default starting from 1.5.0. You may enable it by</p>
<ol>
  <li>setting data source option <code>mergeSchema</code> to <code>true</code> when reading Parquet files (as shown in the examples below), or</li>
  <li>setting the global SQL option <code>spark.sql.parquet.mergeSchema</code> to <code>true</code>.</li>
</ol>